In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URL = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URL)


2025/10/26 09:49:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/26 09:49:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [2]:
client.search_experiments()


[<Experiment: artifact_location='/home/yamraj/Projects/mloops_projects/mloops_self_paced/03-training/mlruns/2', creation_time=1761400939349, experiment_id='2', last_update_time=1761400939349, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/yamraj/Projects/mloops_projects/mloops_self_paced/03-training/mlruns/1', creation_time=1760720322046, experiment_id='1', last_update_time=1760720322046, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1760706299829, experiment_id='0', last_update_time=1760706299829, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from mlflow.entities import ViewType
runs=client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 40",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print (f"run id: {run.info.run_id} | rmse: {run.data.metrics['rmse']:.4f}   ")

run id: b780dc4b47244c52a3f41663d7b21498 | rmse: 39.7519   
run id: 071d9caa9dea4ea9bebbcb0339237478 | rmse: 39.7551   
run id: 7e2ad81587734fc1b25eb14b5a951d4f | rmse: 39.7551   
run id: 4a91ff00e6a74b72b85ea9cff8dc354b | rmse: 39.7551   
run id: a3418a68a5964c58a6f0c92726d12eb2 | rmse: 39.7748   


In [6]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URL)


In [7]:
import mlflow

run_id = "b780dc4b47244c52a3f41663d7b21498"  # removed trailing space
model_uri = f"runs:/{run_id}/models_mlflow"  # correct artifact path

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-experiment")


2025/10/26 09:50:18 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/26 09:50:18 INFO mlflow.store.db.utils: Updating database tables


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-experiment' already exists. Creating a new version of this model...
2025/10/26 09:50:19 WARNING mlflow.tracking._model_registry.fluent: Run with id b780dc4b47244c52a3f41663d7b21498 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-71d20996b83c4763b17924c3971825bb instead
Created version '2' of model 'nyc-taxi-experiment'.


<ModelVersion: aliases=[], creation_timestamp=1761451519151, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761451519151, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [8]:
model_uri

'runs:/b780dc4b47244c52a3f41663d7b21498/models_mlflow'

registered_models = client.search_registered_models()
print(registered_models)


In [9]:
registered_models = client.search_registered_models()
print(registered_models)


[<RegisteredModel: aliases={}, creation_timestamp=1761405083082, deployment_job_id=None, deployment_job_state=None, description=None, last_updated_timestamp=1761451519151, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1761451519151, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761451519151, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='nyc-taxi-experiment', tags={}>, <RegisteredModel: aliases={}, creation_timestamp=1761404985520, deployment_job_id=None, deployment_job_state=None, description=None, last_updated_timestamp=1761404985520, latest_versions=[], name='nyc-taxi-experiment ', tags={}>, <RegisteredModel: aliases={'model_urimodelsnyc-taxi-experimentmodelsproduction': 1}, creation_timestamp=1761390071642, depl

In [10]:

model_name = "nyc-taxi-experiment"
latest_versions=client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version},stage: {version.current_stage}")    

version: 2,stage: None


/tmp/ipykernel_5244/2065705576.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions=client.get_latest_versions(name=model_name)


In [20]:
model_version=1
new_stage="Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_5244/421308582.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1761405542348, current_stage='Staging', deployment_job_state=None, description='the model version1 was transtitioned to Staging on 2025-10-26 10:07:09.867095', last_updated_timestamp=1761452646390, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [21]:
from datetime import datetime
date=datetime.today()


client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"the model version{model_version} was transtitioned to {new_stage} on {date}"

)

<ModelVersion: aliases=[], creation_timestamp=1761405542348, current_stage='Staging', deployment_job_state=None, description='the model version1 was transtitioned to Staging on 2025-10-26 10:09:08.427785', last_updated_timestamp=1761452648442, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [23]:
from sklearn.metrics import mean_squared_error

def read_dataframe(filename):
    import pandas as pd
    df = pd.read_parquet(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    return df   
def preprocess(df,dv):
    df['PU_DO']=df['PULocationID']+'_'+df['DOLocationID']
    categorical = ['PU_DO'] 
    numerical = ['trip_distance']   
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(stage,X_test,y_test):
    model=mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred=model.predict(X_test)
    return {"rmse": mean_squared_error(y_test,y_pred,squared=False )}


    
    

In [28]:
df=read_dataframe('green_tripdata_2021-03.parquet')

In [29]:
client.download_artifacts(run_id=run_id,path='preprocessor',dst_path='.')

'/home/yamraj/Projects/mloops_projects/mloops_self_paced/03-training/preprocessor'

In [31]:
import pickle
with open('preprocessor/preprocessor.b','rb') as f_in:
    dv=pickle.load(f_in)    

In [32]:
X_test=preprocess(df,dv)

In [33]:
target = 'duration'
y_test = df[target].values

In [46]:
import inspect
print(inspect.signature(test_model))


(stage, X_test, y_test)


In [50]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred)}


In [52]:
from mlflow import MlflowClient

client = MlflowClient()
for rm in client.search_registered_models():
    print(rm.name)


nyc-taxi-experiment
nyc-taxi-experiment 
yamraj_model


In [53]:
from mlflow import MlflowClient

client = MlflowClient()

for name in ["nyc-taxi-experiment", "yamraj_model"]:
    print(f"\nModel: {name}")
    versions = client.search_model_versions(f"name='{name}'")
    for v in versions:
        print(f"  Version: {v.version}, Stage: {v.current_stage}, Run ID: {v.run_id}")



Model: nyc-taxi-experiment
  Version: 1, Stage: Staging, Run ID: b780dc4b47244c52a3f41663d7b21498
  Version: 2, Stage: None, Run ID: b780dc4b47244c52a3f41663d7b21498

Model: yamraj_model
  Version: 3, Stage: None, Run ID: 
  Version: 2, Stage: None, Run ID: 
  Version: 1, Stage: None, Run ID: 


In [54]:
from mlflow import MlflowClient

client = MlflowClient()

# Promote version 1 to Production
client.transition_model_version_stage(
    name="nyc-taxi-experiment",
    version=1,
    stage="Production",
    archive_existing_versions=False
)


/tmp/ipykernel_5244/1119790774.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1761405542348, current_stage='Production', deployment_job_state=None, description='the model version1 was transtitioned to Staging on 2025-10-26 10:09:08.427785', last_updated_timestamp=1761463005783, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [55]:
%time test_model("nyc-taxi-experiment", "Production", X_test, y_test)


CPU times: user 1min 8s, sys: 526 ms, total: 1min 9s
Wall time: 22.9 s


{'rmse': 39.164489406755855}

In [57]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)



/tmp/ipykernel_5244/2761509898.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1761405542348, current_stage='Production', deployment_job_state=None, description='the model version1 was transtitioned to Staging on 2025-10-26 10:09:08.427785', last_updated_timestamp=1761463377937, metrics=None, model_id=None, name='nyc-taxi-experiment', params=None, run_id='b780dc4b47244c52a3f41663d7b21498', run_link=None, source='models:/m-71d20996b83c4763b17924c3971825bb', status='READY', status_message=None, tags={}, user_id=None, version=1>